In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset

c:\Users\Admin\anaconda3\envs\nlptv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\anaconda3\envs\nlptv\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


# Load Dataset

In [2]:
data_path = "../data/jutsus.jsonl"
df = pd.read_json(data_path, lines=True)
df.head()

,jutsu_name,jutsu_type,jutsu_description
0,10 Hit Combo,Taijutsu,Lars punches the opponent before striking them...
1,Arm Growth Technique,Ninjutsu,"From the shoulders, the user is able to create..."
2,Armour-Eater,Ninjutsu,"The user melts pieces of their armour, creatin..."
3,Armageddon Countdown Clock,Ninjutsu,"After performing the hand seal, a special cloc..."
4,Arm of Shukaku,Ninjutsu,Gaara engulfs himself in a sand barrier as Shu...


In [3]:
def simplify_jutsu(jutsu):
    if "Genjutsu" in jutsu:
        return "Genjutsu"
    if "Ninjutsu" in jutsu:
        return "Ninjutsu"
    if "Taijutsu" in jutsu:
        return "Taijutsu"

In [4]:
df['jutsu_type_simplified'] = df['jutsu_type'].apply(simplify_jutsu)

In [5]:
df.head()

,jutsu_name,jutsu_type,jutsu_description,jutsu_type_simplified
0,10 Hit Combo,Taijutsu,Lars punches the opponent before striking them...,Taijutsu
1,Arm Growth Technique,Ninjutsu,"From the shoulders, the user is able to create...",Ninjutsu
2,Armour-Eater,Ninjutsu,"The user melts pieces of their armour, creatin...",Ninjutsu
3,Armageddon Countdown Clock,Ninjutsu,"After performing the hand seal, a special cloc...",Ninjutsu
4,Arm of Shukaku,Ninjutsu,Gaara engulfs himself in a sand barrier as Shu...,Ninjutsu


In [6]:
df['jutsu_type_simplified'].value_counts()

jutsu_type_simplified
Ninjutsu    2265
Taijutsu     398
Genjutsu     101
Name: count, dtype: int64

In [7]:
df['text'] = df['jutsu_name'] + ". " + df['jutsu_description']
df['jutsus'] = df['jutsu_type_simplified']
df = df[['text', 'jutsus']]
df = df.dropna()

In [8]:
df.head()

,text,jutsus
0,10 Hit Combo. Lars punches the opponent before...,Taijutsu
1,"Arm Growth Technique. From the shoulders, the ...",Ninjutsu
2,Armour-Eater. The user melts pieces of their a...,Ninjutsu
3,Armageddon Countdown Clock. After performing t...,Ninjutsu
4,Arm of Shukaku. Gaara engulfs himself in a san...,Ninjutsu


In [9]:
from bs4 import BeautifulSoup
class Cleaner():
    def __init__(self):
        pass 
    
    def put_line_breaks(self, text):
        return text.replace("<\p>", "<\p>\n")
    
    def remove_html_tags(self, text):
        clean_text = BeautifulSoup(text, "lxml").text
        return clean_text

    def clean(self, text):
        text = self.put_line_breaks(text)
        text = self.remove_html_tags(text)
        text = text.strip()
        return text

In [10]:
text_column_name = 'text'
label_column_name = "jutsus"

In [11]:
# Clean Text
cleaner = Cleaner()
df['text_cleaned'] = df[text_column_name].apply(cleaner.clean)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13992\3655626712.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_text = BeautifulSoup(text, "lxml").text


In [12]:
df.head(2)

,text,jutsus,text_cleaned
0,10 Hit Combo. Lars punches the opponent before...,Taijutsu,10 Hit Combo. Lars punches the opponent before...
1,"Arm Growth Technique. From the shoulders, the ...",Ninjutsu,"Arm Growth Technique. From the shoulders, the ..."


In [13]:
# Encode Labels 
le = preprocessing.LabelEncoder()
le.fit(df[label_column_name].tolist())

LabelEncoder()

In [14]:
label_dict = {index:label_name for index, label_name in enumerate(le.__dict__['classes_'].tolist())}
label_dict

{0: 'Genjutsu', 1: 'Ninjutsu', 2: 'Taijutsu'}

In [15]:
df['label'] = le.transform(df[label_column_name].tolist())

In [16]:
df.head()

,text,jutsus,text_cleaned,label
0,10 Hit Combo. Lars punches the opponent before...,Taijutsu,10 Hit Combo. Lars punches the opponent before...,2
1,"Arm Growth Technique. From the shoulders, the ...",Ninjutsu,"Arm Growth Technique. From the shoulders, the ...",1
2,Armour-Eater. The user melts pieces of their a...,Ninjutsu,Armour-Eater. The user melts pieces of their a...,1
3,Armageddon Countdown Clock. After performing t...,Ninjutsu,Armageddon Countdown Clock. After performing t...,1
4,Arm of Shukaku. Gaara engulfs himself in a san...,Ninjutsu,Arm of Shukaku. Gaara engulfs himself in a san...,1


In [17]:
test_size = 0.2
df_train, df_test = train_test_split(df, 
                                     test_size=test_size, 
                                     stratify=df['label'],)

In [18]:
df_train['jutsus'].value_counts()

jutsus
Ninjutsu    1812
Taijutsu     318
Genjutsu      81
Name: count, dtype: int64

In [19]:
model_name = "distilbert/distilbert-base-uncased"

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\Admin\anaconda3\envs\nlptv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Admin\anaconda3\envs\nlptv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Py

In [21]:
def preprocess_function(tokenizer,examples):
    return tokenizer(examples['text_cleaned'],truncation=True)

In [22]:
# Conver Pandas to a hugging face dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

# tokenize the dataset
tokenized_train = train_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)
tokenized_test = test_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)

Map: 100%|██████████| 553/553 [00:00<00:00, 8906.75 examples/s]
